In [1]:
from bs4 import BeautifulSoup
import re
import xml.etree.ElementTree as ET
import csv

In [2]:
tei_doc = 'C:/Users/wsmok/Documents/Scrapy/LHI/LHIQ12019.pdf.xml'
with open(tei_doc, 'rb') as tei:
    soup = BeautifulSoup(tei, 'lxml')

In [3]:
wroot = ET.Element('Pdf')

In [4]:
datalist = []

In [5]:
for each in soup.find_all('page'):
    pagenumber = each.get('number')
    #print(pagenumber)
    #for found in each.findAll('text',string=re.compile(r"([\d]{1,2}/[\d]{1,2}/[\d]{4})|([0]{3})|(CONDENSED CONSOLIDATED STATEMENT)|(STATEMENT OF)")):
    sub = ET.SubElement(wroot, 'Page', Number = pagenumber)
    keypage = False
    for found in each.findAll('text'):
        itemlist = []
        position = int(found.get('left')) + int(found.get('width'))
        itemlist.append(pagenumber)
        itemlist.append(found.get('top'))
        itemlist.append(position)
        itemlist.append(found.get('left'))
        itemlist.append(found.text)
        datalist.append(itemlist)
        #print(itemlist)
        #writer.writerow(itemlist)        
        if(found.text is not None and re.search(r"(CONDENSED CONSOLIDATED STATEMENT)|(STATEMENT OF)", found.text.strip())):
            #print(position,found.text)
            keypage = True
            ET.SubElement(sub, 'Title').text = found.text
            sub.set("Key", "Yes")
        if(keypage is True and found.text is not None and re.search(r"([0]{3})", found.text.strip())):
            #print(position,found.text)
            ET.SubElement(sub, 'Column', EndPos = str(position), StartPos= found.get('left')).text = found.text
    if(keypage is False):
        sub.set("Key", "No")

In [6]:
type(datalist)

list

In [7]:
with open("output.csv", "w", newline ='') as f:
    writer = csv.writer(f)
    writer.writerows(datalist)

In [8]:
tree = ET.ElementTree(wroot)
tree.write("filename200121Q1.xml")